In [4]:
import json
import bucket_ops
gcs = bucket_ops.GCSBucketManager(bucket_name="clinic_sim")


In [ ]:
import requests

BASE_URL = "http://localhost:8000"
PATIENT_ID = "P0001"  # <--- REPLACE THIS with an actual ID from your generation step

endpoint = f"{BASE_URL}/chat/{PATIENT_ID}/reset"

response = requests.post(endpoint, json={"patient_id": PATIENT_ID})
response.json()


In [14]:
import requests

BASE_URL = "http://localhost:8000"
PATIENT_ID = "P0001"  # <--- REPLACE THIS with an actual ID from your generation step

endpoint = f"{BASE_URL}/chat/{PATIENT_ID}"

response = requests.get(endpoint)
response.json()


{'conversation': [{'sender': 'admin',
   'message': 'Hello, this is Linda the Hepatology Clinic admin desk. How can I help you today?'},
  {'sender': 'patient',
   'message': 'Hello, I would like to book an appointment with the hepatologist.'},
  {'message': 'Thank you. Please confirm your details in this form.',
   'action_type': 'SEND_FORM',
   'form_request': {'form_id': 'INTAKE_HEP_123', 'fields': {}},
   'sender': 'admin'}]}

In [11]:
import requests
import json
import sys

# Configuration
BASE_URL = "http://localhost:8000"
# REPLACE with a patient_id that actually exists in your data
DEFAULT_PATIENT_ID = "P0001" 
DEFAULT_MESSAGE = "Hello, I would like to book an appointment with the hepatologist."

def test_send_message(patient_id, message):
    endpoint = f"{BASE_URL}/chat"
    
    payload = {
        "patient_id": patient_id,
        "patient_message": message
    }

    print(f"🔵 sending POST request to: {endpoint}")
    print(f"📤 Payload: {json.dumps(payload, indent=2)}")
    print("-" * 40)

    try:
        response = requests.post(endpoint, json=payload)
        
        # Check success
        if response.status_code == 200:
            return response
            
        elif response.status_code == 404:
            print("❌ FAILED (404 Not Found)")
            print(f"Reason: {response.json().get('detail')}")
            print("Hint: This usually means the patient_id folder or chat history file doesn't exist yet.")
            
        else:
            print(f"❌ FAILED ({response.status_code})")
            print(response.text)

    except requests.exceptions.ConnectionError:
        print("❌ CRITICAL ERROR: Could not connect to server.")
        print("Is the server running? Try: python server.py")


    
response_chat = test_send_message(DEFAULT_PATIENT_ID, DEFAULT_MESSAGE)

🔵 sending POST request to: http://localhost:8000/chat
📤 Payload: {
  "patient_id": "P0001",
  "patient_message": "Hello, I would like to book an appointment with the hepatologist."
}
----------------------------------------


In [12]:
response_chat.json()

{'patient_id': 'P0001',
 'nurse_response': {'message': 'Thank you. Please confirm your details in this form.',
  'action_type': 'SEND_FORM',
  'form_request': {'form_id': 'INTAKE_HEP_123', 'fields': {}}},
 'status': 'success'}

In [6]:
raw_data = json.loads(gcs.read_file_as_string(f"patient_data/P0001/raw_data.json"))
raw_data

{'encounter_reports': [{'file': 'encounter_report_0_2026-01-19.txt',
   'date_time': '2026-01-19',
   'encounter_report_text': '   SPRINGFIELD FAMILY CLINIC - URGENT CARE\n   456 Health Avenue, Springfield\n   ----------------------------------------\n   VISIT SUMMARY REPORT\n   Date: 2026-01-19\n\n   PATIENT: Arthur Pendelton (Male, 54)\n   ID: #003\n\n   REASON FOR VISIT:\n   Return of ascites, confusion, and poor medication compliance with recent alcohol relapse.\n\n   CLINICAL NOTES:\n   Patient presents to his GP feeling significantly unwell, reporting a return of his abdominal distension over the past week, stating his trousers are "too tight again." He also notes increased episodes of confusion and forgetfulness, describing it as "my brain\'s gone all fuzzy." He admits to relapsing into heavier alcohol consumption over the last two weeks, averaging around 8-10 units per day, finding it difficult to stop. He reports feeling nauseous and has had a poor appetite. He hasn\'t been ta

In [4]:
file_inventory = json.loads(gcs.read_file_as_string(f"patient_data/P0001/pre_consultation_chat.json"))

In [ ]:
res = {
    "conversation" : [
            {
            'sender': 'admin',
            'message': 'Hello, this is Linda the Hepatology Clinic admin desk. How can I help you today?'
            }
    ]
}
gcs.create_file_from_string(json.dumps(res, indent=4), f"patient_data/P0001/pre_consultation_chat.json", content_type="application/json")

Content uploaded to patient_data/P0001/pre_consultation_chat.json.


True

In [5]:
file_inventory

{'conversation': [{'sender': 'admin',
   'message': 'Hello, thank you for contacting the General Hepatology Clinic. How can I help you today?'},
  {'sender': 'patient',
   'message': "Alright, I need to book an appointment with the specialist. I'm not feeling too good."},
  {'sender': 'admin',
   'message': 'I can help you with that. To confirm your identity, could you please provide your full name, date of birth, and current address?'},
  {'sender': 'patient',
   'message': 'Arthur Pendelton. Born 15th March 1970. I live at Flat 3, 12 Elm Street, Manchester.'},
  {'sender': 'admin',
   'message': "Thank you, Mr. Pendelton. Could you please describe your main symptoms and how long you've been experiencing them?"},
  {'sender': 'patient',
   'message': "Well, me belly's massive, looks like I'm gonna burst. And I'm feelin' proper confused, like I'm not myself. My eyes have gone yellow too. It's been getting worse for a few weeks now, especially the confusion, that's only been a few days.

In [2]:
def group_labs_by_date(labs_data):

        grouped = {}

        for item in labs_data:
            # Extract static metadata for this biomarker
            biomarker_name = item.get("biomarker")
            unit = item.get("unit")
            ref_min = item.get("referenceRange", {}).get("min")
            ref_max = item.get("referenceRange", {}).get("max")

            # Iterate through the time-series values
            for measurement in item.get("values", []):
                timestamp = measurement.get("t")
                value = measurement.get("value")

                # Initialize the date group if it doesn't exist
                if timestamp not in grouped:
                    grouped[timestamp] = {
                        "date_time": timestamp,
                        "labs": []
                    }

                # Determine Flag (High/Low/Normal)
                flag = "NORMAL"
                if ref_min is not None and ref_max is not None:
                    if value < ref_min:
                        flag = "LOW"
                    elif value > ref_max:
                        flag = "HIGH"

                # Add this specific result to the group
                grouped[timestamp]["labs"].append({
                    "biomarker": biomarker_name,
                    "value": value,
                    "unit": unit,
                    "reference_range": f"{ref_min} - {ref_max}",
                    "flag": flag
                })

        # Convert dictionary to a list and sort by date (Newest last)
        # ISO 8601 dates (YYYY-MM-DD) sort correctly as strings
        sorted_results = sorted(list(grouped.values()), key=lambda x: x['date_time'])
        
        return sorted_results

In [3]:
with open("output/P0001/labs.json", "r", encoding="utf-8") as f:
    labs = json.load(f)

In [4]:
labs_data = group_labs_by_date(labs)

In [8]:
len(labs_data)
for i, lab_ in enumerate(labs_data):
    with open(f"output/P0001/lab{i+1}.json", "w", encoding="utf-8") as f:
        json.dump(lab_, f, indent=4)